## Ukázka zpracování dat z SQLite databáze

Nejprve naimportujeme knihovny a potřebné moduly. Pokud je třeba, přidáme cestu k modulu pywsdp do PATH.

In [1]:
import os
import sys

library_path = os.path.abspath(os.path.join("../../"))
if library_path not in sys.path:
    sys.path.append(library_path)

from pywsdp.modules import CtiOS
from pywsdp.modules.CtiOS import OutputFormat


Prihlasime se k modulu CtiOS. Pro tento ucel vyuzijeme testovaci pristupove udaje do DP.

In [2]:
creds_test = ["WSTEST", "WSHESLO"]
ctios = CtiOS(creds_test, trial=True)

Urcime si cesty k vstupni databazi a vystupnimu souboru, kam budeme chtit vyslednou updatovanou databazi ulozit.

In [3]:
db_path = os.path.abspath(
    os.path.join(library_path, "tests", "data", "input", "ctios_template.db")
)

vystupni_adresar = os.path.abspath(
    os.path.join(library_path, "tests", "data", "output")
)

# create output directory if not exists
if os.path.exists(vystupni_adresar) == False:
    os.mkdir(vystupni_adresar)



Nacteme identifikatory z databaze (omezime pocet nactenych identifikatoru selectem).

In [4]:
parametry_ctiOS_db = ctios.nacti_identifikatory_z_db(db_path, "SELECT * FROM OPSUB order by ID LIMIT 5")

Posleme pozadavek a vypiseme si statistiku zpracovanych identifikatoru.

In [5]:
slovnik, slovnik_chybnych = ctios.posli_pozadavek(parametry_ctiOS_db)

ctiOS        - INFO     -  
ctiOS        - INFO     - Požadovaná akce byla úspěšně provedena.
ctiOS        - INFO     - POSIDENT +lQ0Ne+Jhd5K4uvds1/KT8qqJcoA/eJ5OloIoqvCQi04X93kbmO62f/i2fqJgk62odrpl2ye04ue7tlKR93hckH4u46kVMoHQWmYF/rBWMA= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT 00ep7V5fFBZoFMeOfcAOPMRtu0G9svLcsDyC/fYkJk3EbFIqbw2kpRaC1jjS1V8bjacEwrp/SdxXZXdtwDzFj3vKzXvHF7k7HiDpVtv6fPAUIS/cKLcyZ7viQd7pE5FE USPESNE STAZEN
ctiOS        - INFO     - POSIDENT /AOrBGOFR+4GtqsAvOXC/3kKd0OCEPBVRfEsIPHeW3CoGcU1dio8gmrulsH5/KEvySRvbRT2hlDOXICKukDLldMgNY5z+yASe8F1rqBlKug= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT +VKfYaKhJDrRHUWeS2e7xO7g+HhFQapZMPa5kamaF/wSOs6r3xEPEVFhtUlWqcXO9GTopZEjqpsTbsKhZmN7VxYJSxvgcgTWMGdxYkQVCaU= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT /ipU68hnjrq1J7ZDi4JBaAyBiKjxZzpGZSVIfv8yFMKSKy3PpT/lba/0c4YCdqvrexVCCAL0MxBbob3A75l2Tnkgw/HHXtm0+abSHVywLEM= USPESNE STAZEN
ctiOS        - INFO     - Celkovy pocet dotazovanych identifikatoru na vstupu: 5
ctiOS     

Ulozime vysledne identifikatory ve forme slovniku do databaze. Dale podle vyplneneho atributu jmeno (v pripade fyzicke osoby) nebo nazvu (v pripade pravnicke osoby nebo SJM) zkontrolujeme pocet aktualizovanych radku.

In [6]:
vystup, vystup_chybnych = ctios.uloz_vystup(
    slovnik, vystupni_adresar, OutputFormat.GdalDb, slovnik_chybnych
)
assert os.path.exists(vystup) == True

import sqlite3
con = sqlite3.connect(vystup)
cur = con.cursor()
for row in cur.execute("SELECT count(*) FROM OPSUB where jmeno is not null or nazev is not null"):
    assert row == (5,)
con.close()

ctiOS        - INFO     - Vystup byl ulozen zde: /home/linduska/pywsdp/tests/data/output/ctios_16_36_58_28_10_2022.db
